In [3]:
import os
import pandas as pd
import requests
import pprint
import fileinput
import sys

this is the bioapps rest api wiki
http://www.bcgsc.ca/data/bioapps-docs/bioapps.api/docs/

In [19]:
URL = 'http://sbs:8100/'
# print(URL)



# Create session object, which can be used for all requests.
request_handle = requests.Session()

# Set up appropriate headers.
headers = {'Content-Type': 'application/json',
           'Accept': 'application/json'}

# Send the POST request, with your username & password, converted to json.
response = request_handle.post(os.path.join(URL, 'session'),
                               json={'username': 'szong',
                                     'password': '####',
                                     },
                               headers=headers)

# Check that the response is OK.
if response.status_code == 200:
     # Add the authentication token to the headers.
     token = response.json().get('token')
     headers.update({'X-Token': token})
else:
     # View error messages.
     print (response.json())

{'status': 'error', 'errors': [{'location': 'body', 'name': 'session', 'description': 'Username or password is incorrect.'}]}


In [4]:
f = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/124_patients_new/124_patients_bam_vcf_20190115.csv'
df = pd.read_csv(f, usecols=['patient','status', 'DNA_lib', 'strelka_snv_vcf', 'strelka_indel_vcf'])
df = df[df.status == 'Malignant']
df.drop('status', axis=1, inplace=True)
df.shape
df.head(2)


f = '/projects/trans_scratch/validations/workspace/szong/Cervical/mutsig2cv/118_patients.txt'
df1 = pd.read_csv(f,header=None)
df1.columns = ['patient']
df1.head()
df1.shape
patients = df1['patient']
patients[:2]

df = df[df.patient.isin(patients)]
df.columns = ['patient', 'DNA_lib', 'indels', 'snvs']
df.shape

(118, 4)

In [5]:
df.head(2)
df.shape

(118, 4)

In [1]:
from pprint import pprint 
vcfs = []
for ix, row in df.iterrows():    
    lib = row['DNA_lib']
    pat = row['patient']
    response = request_handle.get(os.path.join(URL, 'somatic'), 
                                  params={'library': lib, 
#                                           'patient': pat,
                                          'production': True, 
                                          'analysis_type': 'combined_snv'},
#                                           'analysis_type': 'strelka'}, 
                                  headers=headers).json()

#     print(response)
    for item in response:
        vcfs.append([pat, item.get('data_path')])
#         print(item.get('data_path'))
print(vcfs)
of = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/124_patients_new/bioapps_returned_combinedsnv_path.txt'
pd.DataFrame(vcfs).to_csv(of, index=False, header=None)

In [4]:
for patient_name in fileinput.input():

    patient_name = patient_name.rstrip()
    response = request_handle.get(os.path.join(URL, 'somatic'), 
                                  params={'patient': patient_name, 
                                          'production': True, 
                                          'analysis_type': 'strelka'}, 
                                  headers=headers).json()

    print(response)

    # for library in response:
    #     for libcore in library['libcores']:


    # for foobar in response:
    #    keys = foobar.keys()
    #    for key in keys:
    #        print("\t".join([str(key), str(foobar[key])]))
    #
    #    print("\n")

FileNotFoundError: [Errno 2] No such file or directory: '-f'